In [9]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

## Questions and Answers
1. The neural network regression model performed better than the baseline linear regression model, but the difference was relatively small (MSE decreased by ~0.01).
2. The cost function is "adam", or the squared error using stochastic gradient descent.
3. See table under Neural Network Table.
4. From the table, I can conclude that "shallower" neural network can perform as well as some deeper neural networks as the errors for single hidden layer neutral networks were similar to neural networks with 5 layers. Additionally, increasing the number of layers does not always improve predictive skills (ex: 5 layers but only 1 node per layer) but increasing number of nodes per layer does improve preditive skills (ex: 5 layers but 16 nodes per layer).
5. When I changed the activation function from 'relu' to 'logistics', the mean squared error decreased for almost all options of nodes but for (1), (1,16), (1,1,1,1,1), the mean squared error increased.

In [2]:
df = pd.read_csv('Waves_2023.txt', sep='\s+') 

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Thaomy\AppData\Local\Temp\ipykernel_7340\3125395425.py:1: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv('Waves_2023.txt', sep='\s+')


In [5]:
df1 = df[['WVHT', 'DPD', 'APD', 'MWD', 'WTMP']]
df1 = df1[(df1.WVHT < 99) | (df1.DPD < 99) | (df1.APD < 99) | (df1.MWD < 990)]

Xvar = df1[['DPD','APD','MWD','WTMP']].to_numpy()
yvar = np.array(df1['WVHT'])

print(Xvar)
print(yvar)

[[ 12.5    7.69 269.    15.3 ]
 [ 11.76   7.34 272.    15.3 ]
 [ 12.5    7.08 268.    15.3 ]
 ...
 [ 13.33  10.5  268.    16.8 ]
 [ 14.29  10.32 271.    16.8 ]
 [ 12.5   10.54 271.    16.9 ]]
[1.12 1.16 1.2  ... 1.79 1.66 1.7 ]


In [6]:
# I want 75% train, 10% test, 15% validate
X, X_test, y, y_test = train_test_split(Xvar, yvar, test_size=0.10)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.167)

## Baseline: Linear Model

In [8]:
linear = LinearRegression()
linear.fit(X_train, y_train)
y_train_pred = linear.predict(X_train)
y_val_pred = linear.predict(X_val)
y_test_pred = linear.predict(X_test)

print(mean_squared_error(y_train, y_train_pred))
print(mean_squared_error(y_val, y_val_pred))
print(mean_squared_error(y_test, y_test_pred))

0.19821913148311102
0.19187794204327813
0.18237403438330343


## Neural Network

In [11]:
# 3 layers, 16 nodes per layer
mlp = MLPRegressor(hidden_layer_sizes=(16,16,16,))
mlp.fit(X_train, y_train)
y_train_pred = mlp.predict(X_train)
y_val_pred = mlp.predict(X_val)
y_test_pred = mlp.predict(X_test)

print(mean_squared_error(y_train, y_train_pred))
print(mean_squared_error(y_val, y_val_pred))
print(mean_squared_error(y_test, y_test_pred))

0.1846522568122223
0.18008568828014604
0.17407571343180503


## Neural Network Table

In [16]:
nodes = [(1,), 
        (16,), 
        (1,16,), 
        (16,16,), 
        (16,4,16,), 
        (16,16,16,), 
        (16,16,16,16,), 
        (16,32,64,128,),
        (1,1,1,1,1,),
        (100,100,100,100,100,)]

train_error = []
valid_error = []
test_error = []

for i in nodes:
    mlp = MLPRegressor(i)
    mlp.fit(X_train, y_train)
    
    y_train_pred = mlp.predict(X_train)
    y_val_pred = mlp.predict(X_val)
    y_test_pred = mlp.predict(X_test)
    
    train_err = mean_squared_error(y_train, y_train_pred)
    valid_err = mean_squared_error(y_val, y_val_pred)
    test_err = mean_squared_error(y_test, y_test_pred)
    
    train_error.append(train_err)
    valid_error.append(valid_err)
    test_error.append(test_err)

parameter = {'nodes':nodes,
             'train error':train_error,
             'validation error':valid_error,
             'test error':test_error}

In [17]:
mlp_df = pd.DataFrame(parameter)
mlp_df

,nodes,train error,validation error,test error
0,"(1,)",0.199134,0.192134,0.183058
1,"(16,)",0.200608,0.193767,0.183658
2,"(1, 16)",0.272363,0.267042,0.242965
3,"(16, 16)",0.188399,0.182058,0.172183
4,"(16, 4, 16)",0.281110,0.275752,0.251494
5,"(16, 16, 16)",0.207450,0.200777,0.197576
6,"(16, 16, 16, 16)",0.199892,0.193550,0.185355
7,"(16, 32, 64, 128)",0.196994,0.190627,0.184314
8,"(1, 1, 1, 1, 1)",0.281105,0.275817,0.251360
9,"(100, 100, 100, 100, 100)",0.185168,0.178933,0.170727


## Neural Network with Activation: 'Logistic'

In [18]:
nodes = [(1,), 
        (16,), 
        (1,16,), 
        (16,16,), 
        (16,4,16,), 
        (16,16,16,), 
        (16,16,16,16,), 
        (16,32,64,128,),
        (1,1,1,1,1,),
        (100,100,100,100,100,)]

train_error = []
valid_error = []
test_error = []

for i in nodes:
    mlp = MLPRegressor(i, activation='logistic')
    mlp.fit(X_train, y_train)
    
    y_train_pred = mlp.predict(X_train)
    y_val_pred = mlp.predict(X_val)
    y_test_pred = mlp.predict(X_test)
    
    train_err = mean_squared_error(y_train, y_train_pred)
    valid_err = mean_squared_error(y_val, y_val_pred)
    test_err = mean_squared_error(y_test, y_test_pred)
    
    train_error.append(train_err)
    valid_error.append(valid_err)
    test_error.append(test_err)

parameter = {'nodes':nodes,
             'train error':train_error,
             'validation error':valid_error,
             'test error':test_error}

In [20]:
mlp_log_df = pd.DataFrame(parameter)
mlp_log_df

,nodes,train error,validation error,test error
0,"(1,)",0.281166,0.275822,0.251428
1,"(16,)",0.176608,0.174096,0.166459
2,"(1, 16)",0.281354,0.276248,0.251285
3,"(16, 16)",0.170577,0.168337,0.159871
4,"(16, 4, 16)",0.166736,0.163710,0.155672
5,"(16, 16, 16)",0.179082,0.175142,0.166996
6,"(16, 16, 16, 16)",0.167075,0.164143,0.155553
7,"(16, 32, 64, 128)",0.179502,0.175213,0.166881
8,"(1, 1, 1, 1, 1)",0.281109,0.275753,0.251491
9,"(100, 100, 100, 100, 100)",0.172264,0.168129,0.162589
